In [21]:
import os
import json
from pathlib import Path
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from PIL import Image
from tqdm.auto import tqdm


c:\Users\Samri\anaconda3\envs\dlproject\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
# CONFIGURATION CONSTANTS


# --- PATHS ---
DATASET_PATH = "../TestDataSet" 
LABEL_JSON_PATH = os.path.join(DATASET_PATH, "labels_list.json") 
ADV_DATASET_PATH_TASK1 = "../Adversarial_Test_Set_1"  # Output path for adversarial set 1
ADV_DATASET_PATH_TASK2 = "../AdversarialTestSet2_PGD_Refactored"  # Output path for adversarial set 2
ADV_DATASET_PATH_TASK3 = "../Adversarial_Test_Set_3"  # Output path for adversarial set 3

# --- MODEL & DEVICE ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
WEIGHTS = "IMAGENET1K_V1"

# --- PGD ATTACK PARAMETERS ---
EPSILON = 0.02        # Maximum L∞-norm perturbation
PGD_ALPHA = 0.005     # PGD step size
PGD_ITERS = 5         # Number of PGD iterations

# --- DATALOADER PARAMETERS ---
BATCH_SIZE = 32
NUM_WORKERS = 0


In [23]:
with open(LABEL_JSON_PATH, "r") as file:
    label_lines = json.load(file)

true_imagenet_indices = [int(entry.split(":")[0]) for entry in label_lines]


In [24]:
orig_folder = torchvision.datasets.ImageFolder(root=DATASET_PATH, transform=None)

folder_to_imagenet_index = {
    class_name: true_imagenet_indices[idx]
    for idx, class_name in enumerate(orig_folder.classes)
}

orig_folder.samples = [
    (
        image_path,
        folder_to_imagenet_index[os.path.basename(os.path.dirname(image_path))]
    )
    for image_path, _ in orig_folder.samples
]



In [25]:
orig_folder.samples.sort(key=lambda sample: sample[0])
orig_labels = [label for _, label in orig_folder.samples]


In [26]:
class AdvDataset(Dataset):
    def __init__(self, image_dir, true_labels, transform=None):
        self.image_paths = sorted(
            os.path.join(image_dir, filename)
            for filename in os.listdir(image_dir)
            if filename.endswith(".png")
        )
        if len(self.image_paths) != len(true_labels):
            raise ValueError(
                f"Mismatch: {len(self.image_paths)} images vs {len(true_labels)} labels"
            )

        self.labels = true_labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image = Image.open(self.image_paths[index]).convert("RGB")

        if self.transform:
            image = self.transform(image)

        label = self.labels[index]
        return image, label


In [27]:
# --- Device Setup ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [28]:
# --- Load Pretrained Model ---
new_model = torchvision.models.densenet121(weights="IMAGENET1K_V1")
new_model.eval().to(device)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [29]:
# --- Input Normalization ---
imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std  = [0.229, 0.224, 0.225]

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=imagenet_mean, std=imagenet_std)
])

# --- Dataset Paths ---
paths = {
    "Original": DATASET_PATH,
    "FGSM": ADV_DATASET_PATH_TASK1,
    "PGD": ADV_DATASET_PATH_TASK2,
    "Patch": ADV_DATASET_PATH_TASK3,
}

In [30]:
def evaluate(model, data_loader):
    top1_correct = top5_correct = total_samples = 0

    with torch.no_grad():
        for images, labels in tqdm(data_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, top1_preds = outputs.topk(1, dim=1)
            _, top5_preds = outputs.topk(5, dim=1)

            top1_correct += (top1_preds.squeeze() == labels).sum().item()
            top5_correct += sum([labels[i] in top5_preds[i] for i in range(len(labels))])
            total_samples += images.size(0)

    top1_accuracy = 100 * top1_correct / total_samples
    top5_accuracy = 100 * top5_correct / total_samples

    return top1_accuracy, top5_accuracy


In [31]:
evaluation_results = {}

for dataset_name, dataset_path in paths.items():
    if dataset_name == "Original":
        dataset = orig_folder
        dataset.transform = transform
    else:
        dataset = AdvDataset(dataset_path, orig_labels, transform)

    data_loader = DataLoader(dataset, batch_size=32, shuffle=False)

    print(f"\nEvaluating {dataset_name} dataset on DenseNet-121:")
    top1_accuracy, top5_accuracy = evaluate(new_model, data_loader)
    evaluation_results[dataset_name] = (top1_accuracy, top5_accuracy)

    print(f"Top-1: {top1_accuracy:.2f}%   Top-5: {top5_accuracy:.2f}%")



Evaluating Original dataset on DenseNet-121:


100%|██████████| 16/16 [00:02<00:00,  7.90it/s]


Top-1: 74.60%   Top-5: 93.60%

Evaluating FGSM dataset on DenseNet-121:


100%|██████████| 16/16 [00:06<00:00,  2.64it/s]


Top-1: 43.00%   Top-5: 66.40%

Evaluating PGD dataset on DenseNet-121:


100%|██████████| 16/16 [00:06<00:00,  2.48it/s]


Top-1: 39.00%   Top-5: 64.40%

Evaluating Patch dataset on DenseNet-121:


100%|██████████| 16/16 [00:06<00:00,  2.53it/s]

Top-1: 42.00%   Top-5: 65.40%
